In [4]:
import pickle
from operator import itemgetter
import pandas as pd
import plotly.graph_objects as go
import matplotlib
from matplotlib import cm

from hydra.plotly_charts import PlotlyPriceChart, supersample_data
from hydra.utils import split_dupes


# SYMBOLS = ["circle", "square", "diamond-tall", "diamond-wide"]
# COLORS = ["red", "blue", "lime", "goldenrod"]
HEATMAP_COLORS = 'afmhot_r'
filename = "../2019-01-01 - 2020-12-01 1d overlap=0.99 detrend=True.enviro"
chart_settings = dict(trendline=False, ifft_price=False, extrapolated=True)

file = None
with open(filename, "rb") as handle:
    file = pickle.load(handle)
interesting, charts, aggregate, inputs = itemgetter('data', 'charts', 'aggregate', 'inputs')(file)
window = pd.to_timedelta(inputs['window'])
overlap = inputs['overlap']
traceCount = min(int(1.0/(1-overlap)), len(charts))
kept_frequencies = aggregate['values']
# print(kept_frequencies)
complexity = aggregate['normalized']
 

pair = inputs['pair']
startDate = charts[0]['startDate']
endDate = charts[-1]['endDate']
# startDate = "2017-05-15 00:00"
# endDate = "2020-05-16 00:00"
print(startDate, endDate)
chart_data = {i:{
    "trendline":[],
    "ifft_price":[],
    "startDate":[],
    "endDate": []
} for i in range(traceCount)}


for index, chart in enumerate(charts):
    idx = index % traceCount
    chart_data[idx]['trendline'].append(chart['trendline'])
    chart_data[idx]['ifft_price'].append(chart['approximated_price'])
    # val, direction = chart['first_extrapolated']
    # chart_data[idx]['extrapolated'].append(val)
    # chart_data[idx]['extrapolated_marker'].append(chart['direction'])
    # chart_data[idx]['startDate'].append(chart['startDate'])
    # chart_data[idx]['endDate'].append(chart['endDate'])
# print(charts[0])
extrapolated_distance_bucket_size = window * .01
print(extrapolated_distance_bucket_size)
agg_chart_data = pd.DataFrame({
    "extrapolated": [chart['first_extrapolated'][0] for chart in charts],
    "extrapolated_date": [chart['first_extrapolated'][1] for chart in charts],
    "is_up": [chart['first_extrapolated'][2] for chart in charts],
    "distance":[(chart['first_extrapolated'][1] - chart['endDate']).floor(extrapolated_distance_bucket_size) for chart in charts],
    # "distance_":[(chart['first_extrapolated'][1] - chart['endDate']) for chart in charts],
    "text": [f"{chart['endDate']}\n🌊{chart['extrapolated_wavelength']}\n🔊{chart['extrapolated_amplitude']}" for chart in charts]
})

agg_chart_data.set_index('extrapolated_date', inplace=True) 
agg_data_by_distance = {
    name:{
        "up":group.loc[group["is_up"]],
        "down":group.loc[~group["is_up"]]
    } for name, group in agg_chart_data.groupby('distance')
}


extra_up = agg_chart_data.loc[agg_chart_data["is_up"]]
extra_down = agg_chart_data.loc[~agg_chart_data["is_up"]]

    

# print(trendlines)
priceChart = PlotlyPriceChart(pair, startDate, endDate)

def add_trace(data, chartParams={}, interval=1, useIntervals=[], onlySlice=False, alwaysUseData=False, approximateIntervals=False):
    if useIntervals == False:
        priceChart.add_trace( 
            go.Scatter(x=data.index, y=data.values, **chartParams), onlySlice=onlySlice
        )
        return

    supersampled = supersample_data(data, interval, useIntervals, alwaysUseData=alwaysUseData,approximate=approximateIntervals)
    priceChart.add_trace( 
        go.Scatter(x=supersampled[1440].index, y=supersampled[1440].values, **chartParams), 
        data=supersampled, onlySlice=onlySlice
    )

if chart_settings['ifft_price']:
    for index in range(traceCount):
        if index in chart_data:
            data = pd.concat(chart_data[index]['ifft_price'])
            add_trace(data, chartParams={"mode":"lines", "name":f"ifft-{index}"})
if chart_settings['trendline']:
    for index in range(traceCount):
        if index in chart_data:
            data = pd.concat(chart_data[index]['trendline'])
            add_trace(data, chartParams={
                "mode":"markers", 
                "name":f"trend-{index}", 
                # "marker_symbol":SYMBOLS[index]
            })

color = cm.get_cmap(HEATMAP_COLORS, len(agg_data_by_distance))
useIntervals = [
    srate 
    for srate in [1, 5, 15, 60, 720, 1440]
    if pd.Timedelta(f"{srate * 250 / 5}min") < window]
print(useIntervals)

if chart_settings['extrapolated']:
    for num, (key, val) in enumerate(agg_data_by_distance.items()):
        up = val['up']
        down=val['down']
        hex=matplotlib.colors.rgb2hex(color(num))
        add_trace(
            up['extrapolated'], 
            chartParams={
                "mode":"markers", 
                "name":f"xtra-u {num * extrapolated_distance_bucket_size}", 
                "marker_size":10, 
                "marker_symbol":"triangle-up", 
                "marker_color": hex,
                "text": up['text'], 
            },
            onlySlice=True,
            alwaysUseData=True,
            useIntervals=useIntervals
        )
        add_trace(
            down['extrapolated'], 
            chartParams={
                "mode":"markers", 
                "name":f"xtra-d {num * extrapolated_distance_bucket_size}", 
                "marker_size":10, 
                "marker_symbol":"triangle-down", 
                "marker_color": hex,
                "text": down['text'], 
            },
            onlySlice=True,
            alwaysUseData=True,
            useIntervals=useIntervals
        )

y = complexity.index.tolist()
# y = [str(float(val).round(4)) for val in y]
priceChart.add_trace(
    go.Heatmap(z=complexity.values.tolist(), x=complexity.columns.tolist(), y=y, showscale=False), 
    loc=(1,1),
    traceArgs={}
)
    # priceChart.figure.add_vline(x=startDate, line_width=3)

priceChart.render()


2019-01-01 00:00:00 2020-12-01 23:59:00
0 days 00:14:24
[1, 5, 15]


FigureWidget({
    'data': [{'close': array([ 3820.1,  3886.2,  3786.5, ..., 17732.9, 18194.2, 19708.1]),
    …